In [3]:
# import os; import sys
# sys.path.insert(1, "/Users/petermyers/Desktop/New_Code/perfect_flop_poker/src/main")
# from main import main

In [6]:
import os
import sys
import ast
import math
import itertools
import functools
import subprocess
import numpy as np
import pandas as pd
from copy import deepcopy
from collections import defaultdict


# RFI
lj_rfi = "55+,AJo+,KQo,A9s+,K9s+,Q9s+,J9s+,T9s"
hj_rfi = "22+,AJo+,KJo+,QJo,A9s+,K9s+,Q9s+,J9s+,T9s"
co_rfi = "22+,ATo+,KTo+,QTo+,JTo,A5s+,K9s+,Q9s+,J9s+,T9s"
bn_rfi = "22+,A9o+,K9o+,Q9o+,J9o+,T9o,A2s+,K8s+,Q8s+,J8s+,T8s+,97s+,86s+,75s+,64s+,53s+,43s"
sb_rfi = "22+,A5o+,K9o+,Q9o+,J9o+,T9o,A2s+,K8s+,Q8s+,J8s+,T8s+,97s+,86s+,75s+,64s+,53s+,43s"

# vs RFI
vsRFI_bb_vs_lj__raise = "QQ+,AKo,AQs+"
vsRFI_bb_vs_hj__raise = "QQ+,AKo,AJs+,KQs"
vsRFI_bb_vs_co__raise = "JJ+,AKo,AJs+,KQs,QJs,JTs"
vsRFI_bb_vs_bn__raise = "TT+,AQo+,KQo,ATs+,KTs+,QTs+,JTs"
vsRFI_bb_vs_sb__raise = "99+,AQo+,KQo,A9s+,K9s+,Q9s+,J9s+,T9s"

vsRFI_bb_vs_lj__call = "99-22,AJs-A2s,K6s+,Q8s+,J8s+,T8s+,97s+,86s+,76s,65s,54s,AQo-ATo,KTo+,QTo+,JTo"
vsRFI_bb_vs_hj__call = "99-22,A9s-A2s,K5s+,Q7s+,J7s+,T7s+,96s+,86s+,75s+,65s,54s,43s,AQo-ATo,KTo+,QTo+,JTo"
vsRFI_bb_vs_co__call = "88-22,A7s-A2s,KTs-K2s,QTs-Q5s,J9s-J6s,T8s-T6s,97s-96s,85s+,74s+,64s+,53s+,43s,AJo-A7o,A5o,KJo-K9o,Q9o+,J9o+,T9o"
vsRFI_bb_vs_bn__call = "77-22,A5s-A2s,K9s-K2s,Q9s-Q2s,J9s-J5s,T8s-T5s,97s-95s,86s-84s,75s-74s,63s+,53s+,42s+,32s,ATo-A2o,KJo-K7o,Q8o+,J8o+,T8o+,98o,87o"
vsRFI_bb_vs_sb__call = "33-22,A7s-A6s,A2s,K8s-K2s,Q8s-Q2s,J8s-J4s,T7s-T4s,96s-94s,85s-84s,74s-73s,63s-62s,53s-52s,42s+,32s,ATo-A2o,KJo-K5o,Q7o+,J7o+,T7o+,97o+,87o,76o,65o"

# vs RFI
vsRFI_hj_vs_lj__raise = "JJ+,AJs+,KQs,AKo"
vsRFI_co_vs_lj__raise = "JJ+,AJs+,KQs,AKo"
vsRFI_co_vs_hj__raise = "JJ+,AJs+,A5s-A4s,KQs,T9s,AQo+"
vsRFI_bn_vs_lj__raise = "JJ+,AKs,A5s-A2s,AKo"
vsRFI_bn_vs_hj__raise = "JJ+,AJs+,A8s,A5s-A2s,KQs,76s,65s,54s,AKo"
vsRFI_bn_vs_co__raise = "TT+,ATs+,A7s-A2s,KJs+,QJs,JTs,T9s,76s,65s,54s,AJo+,KQo"
vsRFI_sb_vs_lj__raise = "JJ+,ATs+,KQs,AKo"
vsRFI_sb_vs_hj__raise = "JJ+,ATs+,KJs+,QJs,JTs,AQo+"
vsRFI_sb_vs_co__raise = "JJ+,A9s+,A5s-A4s,KJs+,QJs,JTs,T9s,98s,AJo+,KQo"
vsRFI_sb_vs_bn__raise = "55+,A2s+,K9s+,Q9s+,J9s+,T8s+,98s,87s,76s,ATo+,KQo+"

vsRFI_hj_vs_lj__call = "TT-77,ATs,KJs,QJs,JTs,AQo"
vsRFI_co_vs_lj__call = "TT-66,ATs,KJs,QJs,JTs,AQo"
vsRFI_co_vs_hj__call = "TT-55,ATs,KJs,QJs,JTs"
vsRFI_bn_vs_lj__call = "TT-55,AQs-ATs,KTs+,QTs+,JTs,T9s,98s,87s,AQo"
vsRFI_bn_vs_hj__call = "TT-33,ATs-A9s,KJs-KTs,QTs+,JTs,T9s,98s,87s,AQo-AJo,KQo"
vsRFI_bn_vs_co__call = "99-22,A9s-A8s,KTs,QTs,98s,87s"
vsRFI_sb_vs_lj__call = "TT-77,QJs,JTs,AQo"
vsRFI_sb_vs_hj__call = "TT-66,T9s"
vsRFI_sb_vs_co__call = "TT-55,KTs,QTs"
vsRFI_sb_vs_bn__call = ""

# 3b originals (These are what I tried first and what I'll keep the opponent's strategy as)
RFIvs3B_lj_vs_hjco_call = "JJ-77,AQs-AJs,KQs,QJs,JTs"
RFIvs3B_lj_vs_bn_call = "JJ-77,AQs-AJs,KJs+,QJs,JTs,T9s"
RFIvs3B_lj_vs_blinds_call = "JJ-66,AQs-ATs,KJs+,QJs,JTs,T9s"
RFIvs3B_hj_vs_ahead_call = "TT-66,AQs-ATs,KTs+,QTs+,JTs,T9s,AQo"
RFIvs3B_co_vs_bn_call = "TT-66,AQs-A8s,KTs+,QTs+,JTs,T9s,98s,AQo"
RFIvs3B_co_vs_blinds_call = "TT-55,AQs-A8s,KTs+,QTs+,JTs,T9s,98s,87s,AQo"
RFIvs3B_bnsb_vs_ahead_call = "TT-33,AQs-A6s,K9s+,Q9s+,J9s+,T8s+,97s+,87s,76s,65s,54s,ATo+,KJo+,QJo"

# 3b new (These are tightened ranges that I'll use for unprofitable 3b call situations)
new_RFIvs3B_lj_vs_hjco_call = ""
new_RFIvs3B_lj_vs_bn_call = ""

# Input: range
m = {
    "A": 14, "K": 13, "Q": 12, "J": 11, "T": 10, "9": 9,
    "8": 8, "7": 7, "6": 6, "5": 5, "4": 4, "3": 3, "2": 2
}
def range_to_hands(c_range="JJ+,AJs+,KQs,AKo"):
    temp = c_range.split(",")

    pps = []
    pp = temp[0]
    if "+" in pp:
        for i in range(14,m[pp[0]]-1,-1):
            pps.append([i, i])
    elif "-" in pp:
        for i in range(m[pp[0]],m[pp[-1]]-1,-1):
            pps.append([i, i])
    else:
        pps.append([m[pp[0]], m[pp[0]]])

    ss = []
    temp_s = [x for x in temp if "s" in x]
    for s in temp_s:
        if "+" in s:
            for i in range(m[s[0]]-1,m[s[1]]-1,-1):
                ss.append([m[s[0]], i])
        elif "-" in s:
            for i in range(m[s[1]],m[s[5]]-1,-1):
                ss.append([m[s[0]], i])
        else:
            ss.append([m[s[0]], m[s[1]]])

    os = []
    temp_o = [x for x in temp if "o" in x]
    for o in temp_o:
        if "+" in o:
            for i in range(m[o[0]]-1,m[o[1]]-1,-1):
                os.append([m[o[0]], i])
        elif "-" in o:
            for i in range(m[o[1]],m[o[5]]-1,-1):
                os.append([m[o[0]], i])
        else:
            os.append([m[o[0]], m[o[1]]])
    # Output: [[2,2]], [[14,13]], [[14,13]]
        # PP, Suited, Offsuit
    return pps, ss, os


cat1_rankings = ["set", "trips", "two pair", "overpair 9+", "any overpair", "TP J-kicker",
                 "TP K-kicker", "TP any kicker"]
cat2_nonpaired_rankings = ["top pair bad kicker", "middle pair", "bottom pair", "PP below middle pair",
                           "AJ high", "KQ high", "KJ high bdfd", "K8 high bdfd", ]
cat2_paired_rankings = ["Ace high", "PP below top card", "KQ high", "all"]
cat3_rankings = ["FD", "OESD", "Gutshot", "3 to a straight not all from low end",
                 "3 to a straight low end bdfd", "3 to a straight low end",
                 "5 cards within 7 values with bdfd", "Q- high bdfd",
                 "3 cards within 4 values as overcards", "A- bdfd"]
first_cat4_pp_rankings = ["JJ", "TT", "99", "88", "77", "66", "55", "44", "33", "22"]



def my_hands_cat1_level_x_and_above(x):
    result = [[], [], []]
    if x >= 1:
        result[1] += my_hands_s_straight
        result[2] += my_hands_o_straight
    if x >= 2:
        result[0] += my_hands_pp_sets
    if x >= 3:
        result[1] += my_hands_s_trips
        result[2] += my_hands_o_trips
    if x >= 4:
        result[1] += my_hands_s_two_pair
        result[2] += my_hands_o_two_pair
    if x >= 5:
        result[0] += my_hands_pp_overpair_9plus
    if x >= 6:
        result[0] += my_hands_pp_any_overpair
    if x >= 7:
        result[1] += my_hands_s_tp_k_kicker
        result[2] += my_hands_o_tp_k_kicker
    if x >= 8:
        result[1] += my_hands_s_tp_j_kicker
        result[2] += my_hands_o_tp_j_kicker
    if x >= 9:
        result[1] += my_hands_s_tp_any_kicker
        result[2] += my_hands_o_tp_any_kicker

    result[0].sort(reverse=True)
    result[1].sort(reverse=True)
    result[2].sort(reverse=True)
    result[0] = list(k for k,_ in itertools.groupby(result[0]))
    result[1] = list(k for k,_ in itertools.groupby(result[1]))
    result[2] = list(k for k,_ in itertools.groupby(result[2]))

    # Return result
    my_hands_cat1 = result
    return my_hands_cat1

# Performance improvement by filtering out cat1 from hands already, but would also need a copy of hands
def my_hands_cat2_level_x_and_above(x, my_hands_cat1):
    result = [[], [], []]
    if x >= 1:
        # Cat 1
        result[1] += my_hands_s_straight
        result[2] += my_hands_o_straight
        result[0] += my_hands_pp_sets
        result[1] += my_hands_s_trips
        result[2] += my_hands_o_trips
        result[1] += my_hands_s_two_pair
        result[2] += my_hands_o_two_pair
        result[0] += my_hands_pp_overpair_9plus
        result[0] += my_hands_pp_any_overpair
        result[1] += my_hands_s_tp_k_kicker
        result[2] += my_hands_o_tp_k_kicker
        result[1] += my_hands_s_tp_j_kicker
        result[2] += my_hands_o_tp_j_kicker
        result[1] += my_hands_s_tp_any_kicker
        result[2] += my_hands_o_tp_any_kicker

        # Cat 2
        result[1] += my_hands_s_tp_bad_kicker
        result[2] += my_hands_o_tp_bad_kicker
    if x >= 2:
        result[1] += my_hands_s_middle_pair
        result[2] += my_hands_o_middle_pair
    if x >= 3:
        result[0] += my_hands_pp_below_top_pair
    if x >= 4:
        result[1] += my_hands_s_bottom_pair
        result[2] += my_hands_o_bottom_pair
    if x >= 5:
        result[1] += my_hands_s_aj_high
        result[2] += my_hands_o_aj_high
    if x >= 6:
        result[0] += my_hands_pp_below_middle_pair
    if x >= 7:
        result[1] += my_hands_s_kq_high
        result[2] += my_hands_o_kq_high
    if x >= 8:
        result[0] += my_hands_pp_below_bottom_pair
    if x >= 9:
        result[1] += my_hands_s_kj_high
        result[2] += my_hands_o_kj_high
    if x >= 10:
        result[1] += my_hands_s_k8_high
        result[2] += my_hands_o_k8_high

    result[0].sort(reverse=True)
    result[1].sort(reverse=True)
    result[2].sort(reverse=True)
    result[0] = list(k for k,_ in itertools.groupby(result[0]))
    result[1] = list(k for k,_ in itertools.groupby(result[1]))
    result[2] = list(k for k,_ in itertools.groupby(result[2]))

    # Interim
    cat1_unique_pp = [x for (x,y) in my_hands_cat1[0]]
    cat1_unique_s = [x for (x,y) in my_hands_cat1[1]]
    cat1_unique_o = [x for (x,y) in my_hands_cat1[2]]

    # Remove cat1 from these cat2s
    result[0] = [(x,y) for (x,y) in result[0] if x not in cat1_unique_pp]
    result[1] = [(x,y) for (x,y) in result[1] if x not in cat1_unique_s]
    result[2] = [(x,y) for (x,y) in result[2] if x not in cat1_unique_o]

    # Return result
    my_hands_cat2 = result
    return my_hands_cat2

# Performance improvement by filtering out cat1+cat2 from hands already, but would also need a copy of hands
def my_hands_cat3_level_x_and_above(x, my_hands_cat1, my_hands_cat2):
    bdfd_result = [[], [], []]
    other_result = [[], [], []]
    result = [[], [], []]
    if x >= 1:
        other_result[0] += my_hands_pp_fd
        other_result[1] += my_hands_s_fd
        other_result[2] += my_hands_o_fd
    if x >= 2:
        other_result[0] += my_hands_pp_oesd
        other_result[1] += my_hands_s_oesd
        other_result[2] += my_hands_o_oesd
    if x >= 3:
        other_result[0] += my_hands_pp_gutshot
        other_result[1] += my_hands_s_gutshot
        other_result[2] += my_hands_o_gutshot
    if x >= 4:
        other_result[1] += my_hands_s_3_to_straight_not_all_from_low_end
        other_result[2] += my_hands_o_3_to_straight_not_all_from_low_end
    if x >= 5:
        bdfd_result[1] += my_hands_s_3_to_straight_low_end_bdfd
        bdfd_result[2] += my_hands_o_3_to_straight_low_end_bdfd
    if x >= 6:
        other_result[1] += my_hands_s_3_to_straight_low_end
        other_result[2] += my_hands_o_3_to_straight_low_end
    if x >= 7:
        bdfd_result[1] += my_hands_s_5_unique_cards_within_7_values_bdfd
        bdfd_result[2] += my_hands_o_5_unique_cards_within_7_values_bdfd
    if x >= 8:
        bdfd_result[0] += my_hands_pp_q_minus_bdfd
        bdfd_result[1] += my_hands_s_q_minus_bdfd
        bdfd_result[2] += my_hands_o_q_minus_bdfd
    if x >= 9:
        other_result[1] += my_hands_s_lowest_card_is_one_of_3_cards_within_4_values_and_two_overcards
        other_result[2] += my_hands_o_lowest_card_is_one_of_3_cards_within_4_values_and_two_overcards
    if x >= 10:
        bdfd_result[0] += my_hands_pp_a_minus_bdfd
        bdfd_result[1] += my_hands_s_a_minus_bdfd
        bdfd_result[2] += my_hands_o_a_minus_bdfd

    # Remove duplicates within bdfd hands
    bdfd_result[0].sort(reverse=True)
    bdfd_result[1].sort(reverse=True)
    bdfd_result[2].sort(reverse=True)
    bdfd_result[0] = list(k for k,_ in itertools.groupby(bdfd_result[0]))
    bdfd_result[1] = list(k for k,_ in itertools.groupby(bdfd_result[1]))
    bdfd_result[2] = list(k for k,_ in itertools.groupby(bdfd_result[2]))

    # Add all together
    result[0] = bdfd_result[0] + other_result[0]
    result[1] = bdfd_result[1] + other_result[1]
    result[2] = bdfd_result[2] + other_result[2]

    # Reduce with max combos number used and sort
    groupby_dict = defaultdict(int)
    for val in result[0]:
        groupby_dict[tuple(val[0])] += val[1]
    result[0] = [(sorted(list(x), reverse=True),min(y, 6)) for (x,y) in groupby_dict.items()]

    groupby_dict = defaultdict(int)
    for val in result[1]:
        groupby_dict[tuple(val[0])] += val[1]
    result[1] = [(sorted(list(x), reverse=True),min(y, 4)) for (x,y) in groupby_dict.items()]

    groupby_dict = defaultdict(int)
    for val in result[2]:
        groupby_dict[tuple(val[0])] += val[1]
    result[2] = [(sorted(list(x), reverse=True),min(y, 12)) for (x,y) in groupby_dict.items()]

    # Interim
    cat1_unique_pp = [x for (x,y) in my_hands_cat1[0]]
    cat1_unique_s = [x for (x,y) in my_hands_cat1[1]]
    cat1_unique_o = [x for (x,y) in my_hands_cat1[2]]
    cat2_unique_pp = [x for (x,y) in my_hands_cat2[0]]
    cat2_unique_s = [x for (x,y) in my_hands_cat2[1]]
    cat2_unique_o = [x for (x,y) in my_hands_cat2[2]]

    # Remove cat1 and cat2
    result[0] = [(x,y) for (x,y) in result[0] if x not in cat1_unique_pp and x not in cat2_unique_pp]
    result[1] = [(x,y) for (x,y) in result[1] if x not in cat1_unique_s and x not in cat2_unique_s]
    result[2] = [(x,y) for (x,y) in result[2] if x not in cat1_unique_o and x not in cat2_unique_o]

    # Add cat2 hands
    if x >= 11:
        result[1] += [(x,y) for (x,y) in my_hands_s_k8_high if x not in cat1_unique_s and x not in cat2_unique_s]
        result[2] += [(x,y) for (x,y) in my_hands_o_k8_high if x not in cat1_unique_o and x not in cat2_unique_o]
    if x >= 12:
        result[1] += [(x,y) for (x,y) in my_hands_s_kj_high if x not in cat1_unique_s and x not in cat2_unique_s]
        result[2] += [(x,y) for (x,y) in my_hands_o_kj_high if x not in cat1_unique_o and x not in cat2_unique_o]
    if x >= 13:
        result[0] += [(x,y) for (x,y) in my_hands_pp_below_bottom_pair if x not in cat1_unique_pp and x not in cat2_unique_pp]
    if x >= 14:
        result[1] += [(x,y) for (x,y) in my_hands_s_kq_high if x not in cat1_unique_s and x not in cat2_unique_s]
        result[2] += [(x,y) for (x,y) in my_hands_o_kq_high if x not in cat1_unique_o and x not in cat2_unique_o]
    if x >= 15:
        result[0] += [(x,y) for (x,y) in my_hands_pp_below_middle_pair if x not in cat1_unique_pp and x not in cat2_unique_pp]
    # Add cat4 hands
    if x >= 16:
        remaining_cat2_type_hands_pp = [x for (x,y) in my_hands_pp_below_top_pair]
        remaining_cat2_type_hands_s = [x for (x,y) in my_hands_s_aj_high] + [x for (x,y) in my_hands_s_bottom_pair] + [x for (x,y) in my_hands_s_middle_pair] + [x for (x,y) in my_hands_s_tp_bad_kicker]
        remaining_cat2_type_hands_o = [x for (x,y) in my_hands_o_aj_high] + [x for (x,y) in my_hands_o_bottom_pair] + [x for (x,y) in my_hands_o_middle_pair] + [x for (x,y) in my_hands_o_tp_bad_kicker]
        result[0] += [(x, 6) for x in my_hands[0] if x not in cat1_unique_pp and x not in cat2_unique_pp and x not in remaining_cat2_type_hands_pp]
        result[1] += [(x, 4) for x in my_hands[1] if x not in cat1_unique_s and x not in cat2_unique_s and x not in remaining_cat2_type_hands_s]
        result[2] += [(x, 12) for x in my_hands[2] if x not in cat1_unique_o and x not in cat2_unique_o and x not in remaining_cat2_type_hands_o]
    # Add cat2 hands with pairs
    if x >= 17:
        result[1] += [(x,y) for (x,y) in my_hands_s_aj_high if x not in cat1_unique_s and x not in cat2_unique_s]
        result[2] += [(x,y) for (x,y) in my_hands_o_aj_high if x not in cat1_unique_o and x not in cat2_unique_o]
    if x >= 18:
        result[1] += [(x,y) for (x,y) in my_hands_s_bottom_pair if x not in cat1_unique_s and x not in cat2_unique_s]
        result[2] += [(x,y) for (x,y) in my_hands_o_bottom_pair if x not in cat1_unique_o and x not in cat2_unique_o]
    if x >= 19:
        result[0] += [(x,y) for (x,y) in my_hands_pp_below_top_pair if x not in cat1_unique_pp and x not in cat2_unique_pp]
    if x >= 20:
        result[1] += [(x,y) for (x,y) in my_hands_s_middle_pair if x not in cat1_unique_s and x not in cat2_unique_s]
        result[2] += [(x,y) for (x,y) in my_hands_o_middle_pair if x not in cat1_unique_o and x not in cat2_unique_o]
    if x >= 21:
        result[1] += [(x,y) for (x,y) in my_hands_s_tp_bad_kicker if x not in cat1_unique_s and x not in cat2_unique_s]
        result[2] += [(x,y) for (x,y) in my_hands_o_tp_bad_kicker if x not in cat1_unique_o and x not in cat2_unique_o]

    # Reduce with max combos number used and sort
    groupby_dict = defaultdict(int)
    for val in result[0]:
        groupby_dict[tuple(val[0])] = max(groupby_dict[tuple(val[0])], val[1])
    result[0] = [(sorted(list(x), reverse=True),min(y, 6)) for (x,y) in groupby_dict.items()]

    groupby_dict = defaultdict(int)
    for val in result[1]:
        groupby_dict[tuple(val[0])] = max(groupby_dict[tuple(val[0])], val[1])
    result[1] = [(sorted(list(x), reverse=True),min(y, 4)) for (x,y) in groupby_dict.items()]

    groupby_dict = defaultdict(int)
    for val in result[2]:
        groupby_dict[tuple(val[0])] = max(groupby_dict[tuple(val[0])], val[1])
    result[2] = [(sorted(list(x), reverse=True),min(y, 12)) for (x,y) in groupby_dict.items()]

    # Return results
    my_hands_cat3 = result
    return my_hands_cat3


def opponents_hands_cat1_level_x_and_above(x):
    result = [[], [], []]
    if x >= 1:
        result[1] += opponents_hands_s_straight
        result[2] += opponents_hands_o_straight
    if x >= 2:
        result[0] += opponents_hands_pp_sets
    if x >= 3:
        result[1] += opponents_hands_s_trips
        result[2] += opponents_hands_o_trips
    if x >= 4:
        result[1] += opponents_hands_s_two_pair
        result[2] += opponents_hands_o_two_pair
    if x >= 5:
        result[0] += opponents_hands_pp_overpair_9plus
    if x >= 6:
        result[0] += opponents_hands_pp_any_overpair
    if x >= 7:
        result[1] += opponents_hands_s_tp_k_kicker
        result[2] += opponents_hands_o_tp_k_kicker
    if x >= 8:
        result[1] += opponents_hands_s_tp_j_kicker
        result[2] += opponents_hands_o_tp_j_kicker
    if x >= 9:
        result[1] += opponents_hands_s_tp_any_kicker
        result[2] += opponents_hands_o_tp_any_kicker

    result[0].sort(reverse=True)
    result[1].sort(reverse=True)
    result[2].sort(reverse=True)
    result[0] = list(k for k,_ in itertools.groupby(result[0]))
    result[1] = list(k for k,_ in itertools.groupby(result[1]))
    result[2] = list(k for k,_ in itertools.groupby(result[2]))

    # Return result
    opponents_hands_cat1 = result
    return opponents_hands_cat1

# Performance improvement by filtering out cat1 from hands already, but would also need a copy of hands
def opponents_hands_cat2_level_x_and_above(x, opponents_hands_cat1):
    result = [[], [], []]
    if x >= 1:
        # Cat 1
        result[1] += opponents_hands_s_straight
        result[2] += opponents_hands_o_straight
        result[0] += opponents_hands_pp_sets
        result[1] += opponents_hands_s_trips
        result[2] += opponents_hands_o_trips
        result[1] += opponents_hands_s_two_pair
        result[2] += opponents_hands_o_two_pair
        result[0] += opponents_hands_pp_overpair_9plus
        result[0] += opponents_hands_pp_any_overpair
        result[1] += opponents_hands_s_tp_k_kicker
        result[2] += opponents_hands_o_tp_k_kicker
        result[1] += opponents_hands_s_tp_j_kicker
        result[2] += opponents_hands_o_tp_j_kicker
        result[1] += opponents_hands_s_tp_any_kicker
        result[2] += opponents_hands_o_tp_any_kicker

        # Cat 2
        result[1] += opponents_hands_s_tp_bad_kicker
        result[2] += opponents_hands_o_tp_bad_kicker
    if x >= 2:
        result[1] += opponents_hands_s_middle_pair
        result[2] += opponents_hands_o_middle_pair
    if x >= 3:
        result[0] += opponents_hands_pp_below_top_pair
    if x >= 4:
        result[1] += opponents_hands_s_bottom_pair
        result[2] += opponents_hands_o_bottom_pair
    if x >= 5:
        result[1] += opponents_hands_s_aj_high
        result[2] += opponents_hands_o_aj_high
    if x >= 6:
        result[0] += opponents_hands_pp_below_middle_pair
    if x >= 7:
        result[1] += opponents_hands_s_kq_high
        result[2] += opponents_hands_o_kq_high
    if x >= 8:
        result[0] += opponents_hands_pp_below_bottom_pair
    if x >= 9:
        result[1] += opponents_hands_s_kj_high
        result[2] += opponents_hands_o_kj_high
    if x >= 10:
        result[1] += opponents_hands_s_k8_high
        result[2] += opponents_hands_o_k8_high

    result[0].sort(reverse=True)
    result[1].sort(reverse=True)
    result[2].sort(reverse=True)
    result[0] = list(k for k,_ in itertools.groupby(result[0]))
    result[1] = list(k for k,_ in itertools.groupby(result[1]))
    result[2] = list(k for k,_ in itertools.groupby(result[2]))

    # Interim
    cat1_unique_pp = [x for (x,y) in opponents_hands_cat1[0]]
    cat1_unique_s = [x for (x,y) in opponents_hands_cat1[1]]
    cat1_unique_o = [x for (x,y) in opponents_hands_cat1[2]]

    # Remove cat1 from these cat2s
    result[0] = [(x,y) for (x,y) in result[0] if x not in cat1_unique_pp]
    result[1] = [(x,y) for (x,y) in result[1] if x not in cat1_unique_s]
    result[2] = [(x,y) for (x,y) in result[2] if x not in cat1_unique_o]

    # Return result
    opponents_hands_cat2 = result
    return opponents_hands_cat2

# Performance improvement by filtering out cat1+cat2 from hands already, but would also need a copy of hands
def opponents_hands_cat3_level_x_and_above(x, opponents_hands_cat1, opponents_hands_cat2, skip_4_to_10_and_13_to_15=True):
    bdfd_result = [[], [], []]
    other_result = [[], [], []]
    result = [[], [], []]
    if x >= 1:
        other_result[0] += opponents_hands_pp_fd
        other_result[1] += opponents_hands_s_fd
        other_result[2] += opponents_hands_o_fd
    if x >= 2:
        other_result[0] += opponents_hands_pp_oesd
        other_result[1] += opponents_hands_s_oesd
        other_result[2] += opponents_hands_o_oesd
    if x >= 3:
        other_result[0] += opponents_hands_pp_gutshot
        other_result[1] += opponents_hands_s_gutshot
        other_result[2] += opponents_hands_o_gutshot
    if x >= 4 and not skip_4_to_10_and_13_to_15:
        other_result[1] += opponents_hands_s_3_to_straight_not_all_from_low_end
        other_result[2] += opponents_hands_o_3_to_straight_not_all_from_low_end
    if x >= 5 and not skip_4_to_10_and_13_to_15:
        bdfd_result[1] += opponents_hands_s_3_to_straight_low_end_bdfd
        bdfd_result[2] += opponents_hands_o_3_to_straight_low_end_bdfd
    if x >= 6 and not skip_4_to_10_and_13_to_15:
        other_result[1] += opponents_hands_s_3_to_straight_low_end
        other_result[2] += opponents_hands_o_3_to_straight_low_end
    if x >= 7 and not skip_4_to_10_and_13_to_15:
        bdfd_result[1] += opponents_hands_s_5_unique_cards_within_7_values_bdfd
        bdfd_result[2] += opponents_hands_o_5_unique_cards_within_7_values_bdfd
    if x >= 8 and not skip_4_to_10_and_13_to_15:
        bdfd_result[0] += opponents_hands_pp_q_minus_bdfd
        bdfd_result[1] += opponents_hands_s_q_minus_bdfd
        bdfd_result[2] += opponents_hands_o_q_minus_bdfd
    if x >= 9:
        other_result[1] += opponents_hands_s_lowest_card_is_one_of_3_cards_within_4_values_and_two_overcards
        other_result[2] += opponents_hands_o_lowest_card_is_one_of_3_cards_within_4_values_and_two_overcards
    if x >= 10 and not skip_4_to_10_and_13_to_15:
        bdfd_result[0] += opponents_hands_pp_a_minus_bdfd
        bdfd_result[1] += opponents_hands_s_a_minus_bdfd
        bdfd_result[2] += opponents_hands_o_a_minus_bdfd

    # Remove duplicates within bdfd hands
    bdfd_result[0].sort(reverse=True)
    bdfd_result[1].sort(reverse=True)
    bdfd_result[2].sort(reverse=True)
    bdfd_result[0] = list(k for k,_ in itertools.groupby(bdfd_result[0]))
    bdfd_result[1] = list(k for k,_ in itertools.groupby(bdfd_result[1]))
    bdfd_result[2] = list(k for k,_ in itertools.groupby(bdfd_result[2]))

    # Add all together
    result[0] = bdfd_result[0] + other_result[0]
    result[1] = bdfd_result[1] + other_result[1]
    result[2] = bdfd_result[2] + other_result[2]

    # Reduce with max combos number used and sort
    groupby_dict = defaultdict(int)
    for val in result[0]:
        groupby_dict[tuple(val[0])] += val[1]
    result[0] = [(sorted(list(x), reverse=True),min(y, 6)) for (x,y) in groupby_dict.items()]

    groupby_dict = defaultdict(int)
    for val in result[1]:
        groupby_dict[tuple(val[0])] += val[1]
    result[1] = [(sorted(list(x), reverse=True),min(y, 4)) for (x,y) in groupby_dict.items()]

    groupby_dict = defaultdict(int)
    for val in result[2]:
        groupby_dict[tuple(val[0])] += val[1]
    result[2] = [(sorted(list(x), reverse=True),min(y, 12)) for (x,y) in groupby_dict.items()]

    # Interim
    cat1_unique_pp = [x for (x,y) in opponents_hands_cat1[0]]
    cat1_unique_s = [x for (x,y) in opponents_hands_cat1[1]]
    cat1_unique_o = [x for (x,y) in opponents_hands_cat1[2]]
    cat2_unique_pp = [x for (x,y) in opponents_hands_cat2[0]]
    cat2_unique_s = [x for (x,y) in opponents_hands_cat2[1]]
    cat2_unique_o = [x for (x,y) in opponents_hands_cat2[2]]

    # Remove cat1 and cat2
    result[0] = [(x,y) for (x,y) in result[0] if x not in cat1_unique_pp and x not in cat2_unique_pp]
    result[1] = [(x,y) for (x,y) in result[1] if x not in cat1_unique_s and x not in cat2_unique_s]
    result[2] = [(x,y) for (x,y) in result[2] if x not in cat1_unique_o and x not in cat2_unique_o]

    # Add cat2 hands
    if x >= 11 and not skip_4_to_10_and_13_to_15:
        result[1] += [(x,y) for (x,y) in opponents_hands_s_k8_high if x not in cat1_unique_s and x not in cat2_unique_s]
        result[2] += [(x,y) for (x,y) in opponents_hands_o_k8_high if x not in cat1_unique_o and x not in cat2_unique_o]
    if x >= 12 and not skip_4_to_10_and_13_to_15:
        result[1] += [(x,y) for (x,y) in opponents_hands_s_kj_high if x not in cat1_unique_s and x not in cat2_unique_s]
        result[2] += [(x,y) for (x,y) in opponents_hands_o_kj_high if x not in cat1_unique_o and x not in cat2_unique_o]
    if x >= 13 and not skip_4_to_10_and_13_to_15:
        result[0] += [(x,y) for (x,y) in opponents_hands_pp_below_bottom_pair if x not in cat1_unique_pp and x not in cat2_unique_pp]
    if x >= 14 and not skip_4_to_10_and_13_to_15:
        result[1] += [(x,y) for (x,y) in opponents_hands_s_kq_high if x not in cat1_unique_s and x not in cat2_unique_s]
        result[2] += [(x,y) for (x,y) in opponents_hands_o_kq_high if x not in cat1_unique_o and x not in cat2_unique_o]
    if x >= 15 and not skip_4_to_10_and_13_to_15:
        result[0] += [(x,y) for (x,y) in opponents_hands_pp_below_middle_pair if x not in cat1_unique_pp and x not in cat2_unique_pp]
    # Add cat4 hands
    if x >= 16:
        remaining_cat2_type_hands_pp = [x for (x,y) in opponents_hands_pp_below_bottom_pair] + [x for (x,y) in opponents_hands_pp_below_middle_pair] + [x for (x,y) in opponents_hands_pp_below_top_pair]
        remaining_cat2_type_hands_s = [x for (x,y) in opponents_hands_s_k8_high] + [x for (x,y) in opponents_hands_s_kj_high] + [x for (x,y) in opponents_hands_s_kq_high] + [x for (x,y) in opponents_hands_s_aj_high] + [x for (x,y) in opponents_hands_s_bottom_pair] + [x for (x,y) in opponents_hands_s_middle_pair] + [x for (x,y) in opponents_hands_s_tp_bad_kicker]
        remaining_cat2_type_hands_o = [x for (x,y) in opponents_hands_o_k8_high] + [x for (x,y) in opponents_hands_o_kj_high] + [x for (x,y) in opponents_hands_o_kq_high] + [x for (x,y) in opponents_hands_o_aj_high] + [x for (x,y) in opponents_hands_o_bottom_pair] + [x for (x,y) in opponents_hands_o_middle_pair] + [x for (x,y) in opponents_hands_o_tp_bad_kicker]
        result[0] += [(x, 6) for x in opponents_hands[0] if x not in cat1_unique_pp and x not in cat2_unique_pp and x not in remaining_cat2_type_hands_pp]
        result[1] += [(x, 4) for x in opponents_hands[1] if x not in cat1_unique_s and x not in cat2_unique_s and x not in remaining_cat2_type_hands_s]
        result[2] += [(x, 12) for x in opponents_hands[2] if x not in cat1_unique_o and x not in cat2_unique_o and x not in remaining_cat2_type_hands_o]
    # Add cat2 hands with pairs
    if x >= 17:
        result[1] += [(x,y) for (x,y) in opponents_hands_s_aj_high if x not in cat1_unique_s and x not in cat2_unique_s]
        result[2] += [(x,y) for (x,y) in opponents_hands_o_aj_high if x not in cat1_unique_o and x not in cat2_unique_o]
    if x >= 18:
        result[1] += [(x,y) for (x,y) in opponents_hands_s_bottom_pair if x not in cat1_unique_s and x not in cat2_unique_s]
        result[2] += [(x,y) for (x,y) in opponents_hands_o_bottom_pair if x not in cat1_unique_o and x not in cat2_unique_o]
    if x >= 19:
        result[0] += [(x,y) for (x,y) in opponents_hands_pp_below_top_pair if x not in cat1_unique_pp and x not in cat2_unique_pp]
    if x >= 20:
        result[1] += [(x,y) for (x,y) in opponents_hands_s_middle_pair if x not in cat1_unique_s and x not in cat2_unique_s]
        result[2] += [(x,y) for (x,y) in opponents_hands_o_middle_pair if x not in cat1_unique_o and x not in cat2_unique_o]
    if x >= 21:
        result[1] += [(x,y) for (x,y) in opponents_hands_s_tp_bad_kicker if x not in cat1_unique_s and x not in cat2_unique_s]
        result[2] += [(x,y) for (x,y) in opponents_hands_o_tp_bad_kicker if x not in cat1_unique_o and x not in cat2_unique_o]

    # Reduce with max combos number used and sort
    groupby_dict = defaultdict(int)
    for val in result[0]:
        groupby_dict[tuple(val[0])] = max(groupby_dict[tuple(val[0])], val[1])
    result[0] = [(sorted(list(x), reverse=True),min(y, 6)) for (x,y) in groupby_dict.items()]

    groupby_dict = defaultdict(int)
    for val in result[1]:
        groupby_dict[tuple(val[0])] = max(groupby_dict[tuple(val[0])], val[1])
    result[1] = [(sorted(list(x), reverse=True),min(y, 4)) for (x,y) in groupby_dict.items()]

    groupby_dict = defaultdict(int)
    for val in result[2]:
        groupby_dict[tuple(val[0])] = max(groupby_dict[tuple(val[0])], val[1])
    result[2] = [(sorted(list(x), reverse=True),min(y, 12)) for (x,y) in groupby_dict.items()]

    # Return results
    opponents_hands_cat3 = result
    return opponents_hands_cat3





opponent_unraised_strategy = None # To be defined later; changes by flop
opponent_raised_strategy = {
    'cat1': {1: 6, 2: 6, 3: 6, 4: 6, 5: 6, 6: 6, 7: 6},
    'cat2': {1: 3, 2: 5, 3: 6, 4: 6, 5: 7, 6: 7, 7: 7},
    'cat3': {1: 2, 2: 2, 3: 2, 4: 2, 5: 2, 6: 5, 7: 5},
}
opponent_reraised_strategy = {
    'cat1': {1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0},
    'cat2': {1: 3, 2: 3, 3: 3, 4: 3, 5: 3, 6: 3, 7: 3},
    'cat3': {1: 2, 2: 2, 3: 2, 4: 2, 5: 2, 6: 3, 7: 3},
}
opponent_strategy = None
def get_flop_type_number():
    return \
    7 if flop[0] == flop[1] or flop[1] == flop[2] else \
    1 if flop[0] >= 13 and flop[1] >= 13 else \
    2 if flop[0] >= 13 and flop[1] >= 9 else \
    3 if flop[0] >= 13 else \
    4 if flop[0] >= 10 and flop[1] >= 9 else \
    5 if flop[0] >= 10 else \
    6
def get_opponent_situation(bets):
    return \
    "oop_open" if bets == 0 and my_position_ip == True else \
    "oop_vs_cb" if bets == 1 and my_position_ip == True else \
    "oop_vs_br" if bets >= 2 and my_position_ip == True else \
    "ip_vs_c" if bets == 0 and my_position_ip == False else \
    "ip_vs_b" if bets == 1 and my_position_ip == False else \
    "ip_vs_cbr"


#####

all_flops = []
for rank1 in range(14,1,-1):
    for rank2 in range(14,1,-1):
        for rank3 in range(14,1,-1):
            if rank1 >= rank2 and rank2 >= rank3:
                all_flops.append([rank1, rank2, rank3])

# # Start from a spot
# flops = flops[406:]


range_names = ["00 LJ vs BB 3Bet", "01 LJ vs BB Call", "02 LJ vs HJ 3Bet", "03 LJ vs CO 3Bet", "04 LJ vs BN 3Bet", "05 LJ vs SB 3Bet", "06 LJ vs HJ Call", "07 LJ vs CO Call", "08 LJ vs BN Call", "09 LJ vs SB Call", "10 HJ vs BB 3Bet", "11 HJ vs BB Call", "12 HJ vs CO 3Bet", "13 HJ vs BN 3Bet", "14 HJ vs SB 3Bet", "15 HJ vs CO Call", "16 HJ vs BN Call", "17 HJ vs SB Call", "18 CO vs BB 3Bet", "19 CO vs BB Call", "20 CO vs BN 3Bet", "21 CO vs SB 3Bet", "22 CO vs BN Call", "23 CO vs SB Call", "24 BN vs BB 3Bet", "25 BN vs BB Call", "26 BN vs SB 3Bet", "27 SB vs BB 3Bet", "28 SB vs BB Call", "29 BB 3Bet vs LJ", "30 BB Call vs LJ", "31 HJ 3Bet vs LJ", "32 CO 3Bet vs LJ", "33 BN 3Bet vs LJ", "34 SB 3Bet vs LJ", "35 HJ Call vs LJ", "36 CO Call vs LJ", "37 BN Call vs LJ", "38 SB Call vs LJ", "39 BB 3Bet vs HJ", "40 BB Call vs HJ", "41 CO 3Bet vs HJ", "42 BN 3Bet vs HJ", "43 SB 3Bet vs HJ", "44 CO Call vs HJ", "45 BN Call vs HJ", "46 SB Call vs HJ", "47 BB 3Bet vs CO", "48 BB Call vs CO", "49 BN 3Bet vs CO", "50 SB 3Bet vs CO", "51 BN Call vs CO", "52 SB Call vs CO", "53 BB 3Bet vs BN", "54 BB Call vs BN", "55 SB 3Bet vs BN", "56 BB 3Bet vs SB", "57 BB Call vs SB",]
my_ranges = [RFIvs3B_lj_vs_blinds_call, lj_rfi, new_RFIvs3B_lj_vs_hjco_call, new_RFIvs3B_lj_vs_hjco_call, new_RFIvs3B_lj_vs_bn_call, RFIvs3B_lj_vs_blinds_call, lj_rfi, lj_rfi, lj_rfi, lj_rfi, RFIvs3B_hj_vs_ahead_call, hj_rfi, RFIvs3B_hj_vs_ahead_call, RFIvs3B_hj_vs_ahead_call, RFIvs3B_hj_vs_ahead_call, hj_rfi, hj_rfi, hj_rfi, RFIvs3B_co_vs_blinds_call, co_rfi, RFIvs3B_co_vs_bn_call, RFIvs3B_co_vs_blinds_call, co_rfi, co_rfi, RFIvs3B_bnsb_vs_ahead_call, bn_rfi, RFIvs3B_bnsb_vs_ahead_call, RFIvs3B_bnsb_vs_ahead_call, sb_rfi, vsRFI_bb_vs_lj__raise, vsRFI_bb_vs_lj__call, vsRFI_hj_vs_lj__raise, vsRFI_co_vs_lj__raise, vsRFI_bn_vs_lj__raise, vsRFI_sb_vs_lj__raise, vsRFI_hj_vs_lj__call, vsRFI_co_vs_lj__call, vsRFI_bn_vs_lj__call, vsRFI_sb_vs_lj__call, vsRFI_bb_vs_hj__raise, vsRFI_bb_vs_hj__call, vsRFI_co_vs_hj__raise, vsRFI_bn_vs_hj__raise, vsRFI_sb_vs_hj__raise, vsRFI_co_vs_hj__call, vsRFI_bn_vs_hj__call, vsRFI_sb_vs_hj__call, vsRFI_bb_vs_co__raise, vsRFI_bb_vs_co__call, vsRFI_bn_vs_co__raise, vsRFI_sb_vs_co__raise, vsRFI_bn_vs_co__call, vsRFI_sb_vs_co__call, vsRFI_bb_vs_bn__raise, vsRFI_bb_vs_bn__call, vsRFI_sb_vs_bn__raise, vsRFI_bb_vs_sb__raise, vsRFI_bb_vs_sb__call]
opponents_ranges = [vsRFI_bb_vs_lj__raise, vsRFI_bb_vs_lj__call, vsRFI_hj_vs_lj__raise, vsRFI_co_vs_lj__raise, vsRFI_bn_vs_lj__raise, vsRFI_sb_vs_lj__raise, vsRFI_hj_vs_lj__call, vsRFI_co_vs_lj__call, vsRFI_bn_vs_lj__call, vsRFI_sb_vs_lj__call, vsRFI_bb_vs_hj__raise, vsRFI_bb_vs_hj__call, vsRFI_co_vs_hj__raise, vsRFI_bn_vs_hj__raise, vsRFI_sb_vs_hj__raise, vsRFI_co_vs_hj__call, vsRFI_bn_vs_hj__call, vsRFI_sb_vs_hj__call, vsRFI_bb_vs_co__raise, vsRFI_bb_vs_co__call, vsRFI_bn_vs_co__raise, vsRFI_sb_vs_co__raise, vsRFI_bn_vs_co__call, vsRFI_sb_vs_co__call, vsRFI_bb_vs_bn__raise, vsRFI_bb_vs_bn__call, vsRFI_sb_vs_bn__raise, vsRFI_bb_vs_sb__raise, vsRFI_bb_vs_sb__call, RFIvs3B_lj_vs_blinds_call, lj_rfi, RFIvs3B_lj_vs_hjco_call, RFIvs3B_lj_vs_hjco_call, RFIvs3B_lj_vs_bn_call, RFIvs3B_lj_vs_blinds_call, lj_rfi, lj_rfi, lj_rfi, lj_rfi, RFIvs3B_hj_vs_ahead_call, hj_rfi, RFIvs3B_hj_vs_ahead_call, RFIvs3B_hj_vs_ahead_call, RFIvs3B_hj_vs_ahead_call, hj_rfi, hj_rfi, hj_rfi, RFIvs3B_co_vs_blinds_call, co_rfi, RFIvs3B_co_vs_bn_call, RFIvs3B_co_vs_blinds_call, co_rfi, co_rfi, RFIvs3B_bnsb_vs_ahead_call, bn_rfi, RFIvs3B_bnsb_vs_ahead_call, RFIvs3B_bnsb_vs_ahead_call, sb_rfi]
my_position_ips = [True, True, False, False, False, True, False, False, False, True, True, True, False, False, True, False, False, True, True, True, False, True, False, True, True, True, True, False, False, False, False, True, True, True, False, True, True, True, False, False, False, True, True, False, True, True, False, False, False, True, False, True, False, False, False, False, True, True]
opponent_pfrs = [True, False, True, True, True, True, False, False, False, False, True, False, True, True, True, False, False, False, True, False, True, True, False, False, True, False, True, True, False, False, True, False, False, False, False, True, True, True, True, False, True, False, False, False, True, True, True, False, True, False, False, True, True, False, True, False, False, True]
pot_sizes = [18.5, 5.5, 19.5, 19.5, 19.5, 19, 6.5, 6.5, 6.5, 6, 18.5, 5.5, 19.5, 19.5, 19, 6.5, 6.5, 6, 18.5, 5.5, 19.5, 19, 6.5, 6, 18.5, 5.5, 19, 18, 6, 18.5, 5.5, 19.5, 19.5, 19.5, 19, 6.5, 6.5, 6.5, 6, 18.5, 5.5, 19.5, 19.5, 19, 6.5, 6.5, 6, 18.5, 5.5, 19.5, 19, 6.5, 6, 18.5, 5.5, 19, 18, 6]
my_investments = [9, 2.5, 9, 9, 9, 9, 2.5, 2.5, 2.5, 2.5, 9, 2.5, 9, 9, 9, 2.5, 2.5, 2.5, 9, 2.5, 9, 9, 2.5, 2.5, 9, 2.5, 9, 8.5, 2.5, 8, 1.5, 9, 9, 9, 8.5, 2.5, 2.5, 2.5, 2, 8, 1.5, 9, 9, 8.5, 2.5, 2.5, 2, 8, 1.5, 9, 8.5, 2.5, 2, 8, 1.5, 8.5, 8, 2]



In [16]:
my_hands_string = "AA,KK"
opponents_hands_string = "AA,55,44"
final_flop_string = "2c2h3d"

In [17]:
command = "source /Users/petermyers/Documents/pbots_calc-master/venv/bin/activate; /Users/petermyers/Documents/pbots_calc-master/python/calculator.sh {}:{} {}".format(my_hands_string, opponents_hands_string, final_flop_string)
process = subprocess.Popen(command,stdout=subprocess.PIPE, shell=True)
raw_equity = ast.literal_eval(process.communicate()[0].strip().decode("utf-8"))[0][1]
raw_equity





0.7228739002932552

In [18]:
lj_rfi
hj_rfi
co_rfi
bn_rfi
sb_rfi
vsRFI_bb_vs_lj__raise
vsRFI_bb_vs_hj__raise
vsRFI_bb_vs_co__raise
vsRFI_bb_vs_bn__raise
vsRFI_bb_vs_sb__raise
vsRFI_bb_vs_lj__call
vsRFI_bb_vs_hj__call
vsRFI_bb_vs_co__call
vsRFI_bb_vs_bn__call
vsRFI_bb_vs_sb__call
vsRFI_hj_vs_lj__raise
vsRFI_co_vs_lj__raise
vsRFI_co_vs_hj__raise
vsRFI_bn_vs_lj__raise
vsRFI_bn_vs_hj__raise
vsRFI_bn_vs_co__raise
vsRFI_sb_vs_lj__raise
vsRFI_sb_vs_hj__raise
vsRFI_sb_vs_co__raise
vsRFI_sb_vs_bn__raise
vsRFI_hj_vs_lj__call
vsRFI_co_vs_lj__call
vsRFI_co_vs_hj__call
vsRFI_bn_vs_lj__call
vsRFI_bn_vs_hj__call
vsRFI_bn_vs_co__call
vsRFI_sb_vs_lj__call
vsRFI_sb_vs_hj__call
vsRFI_sb_vs_co__call
vsRFI_sb_vs_bn__call
RFIvs3B_lj_vs_hjco_call
RFIvs3B_lj_vs_bn_call
RFIvs3B_lj_vs_blinds_call
RFIvs3B_hj_vs_ahead_call
RFIvs3B_co_vs_bn_call
RFIvs3B_co_vs_blinds_call
RFIvs3B_bnsb_vs_ahead_call

'TT-33,AQs-A6s,K9s+,Q9s+,J9s+,T8s+,97s+,87s,76s,65s,54s,ATo+,KJo+,QJo'

In [19]:
lj_rfi

'55+,A2s+,K9s+,Q9s+,J9s+,T9s,98s,87s,76s,AJo+,KQo+'

In [20]:
import random

In [49]:
ranks = [str(x) for x in list(range(2,10))] + ['T'] + ['J'] + ['Q'] + ['K'] + ['A']
print("".join(sorted([random.choice(ranks), random.choice(ranks), random.choice(ranks)], reverse=True)))
print("Tones: ", random.choice([1] + [2]*13 + [3]*6))
print(random.choice(range_names))


K64
Tones:  2
10 HJ vs BB 3Bet


In [26]:
list(range(2,15))

[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14]